In [1]:
# fare script input - output

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import random

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline
pd.set_option('display.max_columns', None)

df = pd.read_csv('Data/COVID_ES.csv')



In [2]:
df.describe(include = 'all')

,ID,Date,Cases,Cases_New,Type,Age,Sex,Source
count,2786,2786,2.628000e+03,2.628000e+03,2786,2786,2786,2786
unique,1,338,NaN,NaN,6,1,1,2
top,ES,2020-09-08,NaN,NaN,Deaths,Total,Total,JRC
freq,2786,10,NaN,NaN,577,2786,2786,1434
mean,NaN,NaN,2.379300e+05,2.358507e+04,NaN,NaN,NaN,NaN
std,NaN,NaN,4.044157e+05,1.477075e+05,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000e+00,-1.354900e+04,NaN,NaN,NaN,NaN
25%,NaN,NaN,2.045300e+04,0.000000e+00,NaN,NaN,NaN,NaN
50%,NaN,NaN,8.119200e+04,2.900000e+02,NaN,NaN,NaN,NaN
75%,NaN,NaN,2.201772e+05,3.103000e+03,NaN,NaN,NaN,NaN


In [3]:
df.drop(columns = ['ID','Age','Sex'], inplace = True)

In [24]:
contagio = df[(df['Date'] >= '2020-02-24') & (df['Date'] <= '2020-12-25')]
#contagio = df[(df['Date'] >= '2020-01-11') & (df['Date'] <= '2020-03-15')]

In [25]:
contagio

,Date,Cases,Cases_New,Type,Source
132,2020-02-24,0.0,0.0,Active,JHU
133,2020-02-24,2.0,0.0,Confirmed,JHU
134,2020-02-24,0.0,0.0,Deaths,JHU
135,2020-02-24,2.0,0.0,Recovered,JHU
136,2020-02-25,4.0,4.0,Active,JHU
...,...,...,...,...,...
2781,2020-12-23,150376.0,0.0,Recovered,JHU
2782,2020-12-24,1654751.0,12536.0,Active,JHU
2783,2020-12-24,1854951.0,12662.0,Confirmed,JHU
2784,2020-12-24,49824.0,126.0,Deaths,JHU


In [26]:
contagio.describe(include = 'all')

,Date,Cases,Cases_New,Type,Source
count,2654,2.496000e+03,2.496000e+03,2654,2654
unique,305,NaN,NaN,6,2
top,2020-09-08,NaN,NaN,Deaths,JRC
freq,10,NaN,NaN,544,1434
mean,NaN,2.505128e+05,2.483236e+04,NaN,NaN
std,NaN,4.111588e+05,1.514622e+05,NaN,NaN
min,NaN,0.000000e+00,-1.354900e+04,NaN,NaN
25%,NaN,2.713600e+04,2.000000e+00,NaN,NaN
50%,NaN,9.803500e+04,3.370000e+02,NaN,NaN
75%,NaN,2.295400e+05,3.505500e+03,NaN,NaN


In [27]:
contagio['Type'].unique()

array(['Active', 'Confirmed', 'Deaths', 'Recovered', 'Hospitalized',
       'ICU'], dtype=object)

In [28]:
contagio[contagio['Type']=='Active']

,Date,Cases,Cases_New,Type,Source
132,2020-02-24,0.0,0.0,Active,JHU
136,2020-02-25,4.0,4.0,Active,JHU
140,2020-02-26,11.0,7.0,Active,JHU
144,2020-02-27,13.0,2.0,Active,JHU
148,2020-02-28,30.0,17.0,Active,JHU
...,...,...,...,...,...
2759,2020-12-21,1769596.0,10259.0,Active,JRC
2768,2020-12-22,1630007.0,10394.0,Active,JHU
2769,2020-12-22,1781645.0,12049.0,Active,JRC
2778,2020-12-23,1642215.0,12208.0,Active,JHU


In [29]:
contagio[contagio.Date == '2020-10-15']

,Date,Cases,Cases_New,Type,Source
2214,2020-10-15,737445.0,13178.0,Active,JHU
2215,2020-10-15,887821.0,13178.0,Active,JRC
2216,2020-10-15,921374.0,13318.0,Confirmed,JHU
2217,2020-10-15,921374.0,13318.0,Confirmed,JRC
2218,2020-10-15,33553.0,140.0,Deaths,JHU
2219,2020-10-15,33553.0,140.0,Deaths,JRC
2220,2020-10-15,160282.0,1052.0,Hospitalized,JRC
2221,2020-10-15,14424.0,73.0,ICU,JRC
2222,2020-10-15,150376.0,0.0,Recovered,JHU
2223,2020-10-15,NaN,NaN,Recovered,JRC


In [30]:
#preprocessing per semplice sird

contagio = contagio[contagio['Source']=='JHU']
    
#interessante costruire modello con tested (ICU, Hospitalized per SIDARTHE)    

dates = []
infected = []
dead = []
recovered = []

for i in contagio.Date.unique():
    dates.append(contagio[(contagio['Date'] == i) & (contagio['Type'] == 'Active')].Date.item())
    infected.append(contagio[(contagio['Date'] == i) & (contagio['Type'] == 'Active')].Cases.item())
    dead.append(contagio[(contagio['Date'] == i) & (contagio['Type'] == 'Deaths')].Cases.item())
    recovered.append(contagio[(contagio['Date'] == i) & (contagio['Type'] == 'Recovered')].Cases.item())

In [31]:
sird = pd.DataFrame(list(zip(dates, infected, dead, recovered)), columns = ['Date','Infected','Dead','Recovered'])

In [11]:
sird.to_csv('Data/Sird_Spain.csv', index = False)

In [67]:
#sird.to_csv('Data/Sird.csv', index = False)